# Econometrics 1 

## TD 4 - 17/10/2024

In [3]:
# Install packages
if(!require(haven)) install.packages("haven")  
if(!require(car)) install.packages("car")
if(!require(estimatr)) install.packages("estimatr")  
if(!require(lmtest)) install.packages("lmtest")

#Load packages
library(haven)
library(car)
library(estimatr)
library(lmtest)


Le chargement a nécessité le package : haven

Le chargement a nécessité le package : car

Le chargement a nécessité le package : carData

Le chargement a nécessité le package : estimatr

Le chargement a nécessité le package : lmtest

Le chargement a nécessité le package : zoo


Attachement du package : ‘zoo’


Les objets suivants sont masqués depuis ‘package:base’:

    as.Date, as.Date.numeric




0. Make some basic descriptive statistics to apprehend the database.

In [4]:
rm(list=ls()) #Clean workspace
data <- read_dta("hprice1.dta")


####################
#### Question 0 ####
####################

# Some descriptive statistics
summary(data)


     price           assess          bdrms          lotsize          sqrft     
 Min.   :111.0   Min.   :198.7   Min.   :2.000   Min.   : 1000   Min.   :1171  
 1st Qu.:230.0   1st Qu.:253.9   1st Qu.:3.000   1st Qu.: 5733   1st Qu.:1660  
 Median :265.5   Median :290.2   Median :3.000   Median : 6430   Median :1845  
 Mean   :293.5   Mean   :315.7   Mean   :3.568   Mean   : 9020   Mean   :2014  
 3rd Qu.:326.2   3rd Qu.:352.1   3rd Qu.:4.000   3rd Qu.: 8583   3rd Qu.:2227  
 Max.   :725.0   Max.   :708.6   Max.   :7.000   Max.   :92681   Max.   :3880  
    colonial          lprice         lassess         llotsize     
 Min.   :0.0000   Min.   :4.710   Min.   :5.292   Min.   : 6.908  
 1st Qu.:0.0000   1st Qu.:5.438   1st Qu.:5.537   1st Qu.: 8.654  
 Median :1.0000   Median :5.582   Median :5.671   Median : 8.769  
 Mean   :0.6932   Mean   :5.633   Mean   :5.718   Mean   : 8.905  
 3rd Qu.:1.0000   3rd Qu.:5.788   3rd Qu.:5.864   3rd Qu.: 9.058  
 Max.   :1.0000   Max.   :6.586   Max.

First, we would like to know if the assessment (the variable assess in the database) is "rational",
i.e. faithful to the selling price.

1. How can you use a simple linear regression to test that?
Hint: Think about (i) a relevant regression, (ii) a suited multiple/joint test. Write formally
the null and alternative hypotheses of that test using the matrix R and vector b introduced
in Chapter 2.

2. Estimate the regression proposed in Question 1 and perform the test. What do you con-
clude? Perform the corresponding simple bilateral tests (Student t-tests). What do you
remark? Is there any problem here?

In [5]:
####################
#### Question 2 ####
####################

reg1 = lm_robust(price ~ assess, data = data , se_type = "stata")
reg1

# Perform a joint test: Beta_0 = 0 and Beta_1 = 1:
R = diag(2)
b = c(0,1)
lht(reg1, hypothesis.matrix = R, rhs = b, test = "F")[2,]


# => We do reject the hypothesis at levels 1%,5%,10 % > 2.985e-05

# Test that the coefficient of the constant is 0:
R = c(1,0)
b = 0
lht(reg1, hypothesis.matrix = R, rhs = b, test = "F")[2,]

# And that the coefficient of the slope is 1:
R = c(0,1) 
b = 1 
lht(reg1, hypothesis.matrix = R, rhs = b, test = "F")[2,]

               Estimate  Std. Error    t value    Pr(>|t|)    CI Lower
(Intercept) -14.4717880 20.61722468 -0.7019271 4.84620e-01 -55.4574742
assess        0.9755538  0.06797519 14.3516157 1.51747e-24   0.8404236
             CI Upper DF
(Intercept) 26.513898 86
assess       1.110684 86

,Res.Df,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
2,86,2,11.79016,2.984919e-05


,Res.Df,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
2,86,1,0.4927016,0.48462


,Res.Df,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
2,86,1,0.1293362,0.7200029


3. We consider the multiple linear regression of the logarithm of the house price on the area
of the house and the number of bedrooms. Estimate that regression. What is the eﬀect of
adding a bedroom of 150 square feet on the price?

In [6]:
####################
#### Question 3 ####
####################

# Regress lprice on sqrft and bdrms:
reg2 = lm_robust(lprice ~sqrft + bdrms, data=data, se_type = "stata")
reg2
# Effects of exogeneous changes:
as.double(c(0,150,1) %*% reg2$coefficients) # Remember the model is log - level.

               Estimate   Std. Error    t value     Pr(>|t|)      CI Lower
(Intercept) 4.766027333 9.255927e-02 51.4916263 7.292921e-66  4.5819947070
sqrft       0.000379446 5.123428e-05  7.4060965 8.737438e-11  0.0002775785
bdrms       0.028884442 3.191088e-02  0.9051597 3.679393e-01 -0.0345629323
                CI Upper DF
(Intercept) 4.9500599584 85
sqrft       0.0004813135 85
bdrms       0.0923318163 85

[1] 0.08580134

4. Propose a new regression that allows estimating the previous eﬀect of adding a bedroom
of 150 square feet on the price directly. Construct a 95%-level conﬁdence interval on that
eﬀect.
2


In [7]:
####################
#### Question 4 ####
####################

# Another method to get a confidence interval on the effect directly:
# 4.1 - Create the new variable:
data$sqrft150b =   data$sqrft - 150 * data$bdrms
# 4.2 - Run the regression:
reg3 = lm_robust(lprice~sqrft150b + bdrms, data = data, se_type = "stata")
reg3
# 4.3 - Get the coefficient and its interval on the variable bdrms:
reg3$coefficients[3]
# The confidence interval is:
c('low_bound' = reg3$conf.low[3], 'high_bound' = reg3$conf.high[3])



               Estimate   Std. Error   t value     Pr(>|t|)     CI Lower
(Intercept) 4.766027333 9.255927e-02 51.491626 7.292921e-66 4.5819947070
sqrft150b   0.000379446 5.123428e-05  7.406097 8.737438e-11 0.0002775785
bdrms       0.085801342 2.774194e-02  3.092839 2.682170e-03 0.0306429399
                CI Upper DF
(Intercept) 4.9500599584 85
sqrft150b   0.0004813135 85
bdrms       0.1409597435 85

bdrms 
0.08580134

low_bound.bdrms high_bound.bdrms 
      0.03064294       0.14095974